<a href="https://colab.research.google.com/github/jasonix5/DL-UniWA/blob/main/DL_UniWA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Cell 1 ─────────────────────────────────────────────────────────────
!pip install -q "numpy==1.26.4" timm==0.9.12 albumentations \
               scikit-learn pandas matplotlib
import torch, numpy as np
print("Torch:", torch.__version__)   # 2.6.0+cu124
print("NumPy:", np.__version__)      # 1.26.4
print("CUDA :", torch.cuda.is_available())


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 113.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 95.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 102.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Cell 2 ─────────────────────────────────────────────────────────────
!wget -q https://zenodo.org/records/7711810/files/EuroSAT_RGB.zip -O EuroSAT_RGB.zip
!unzip -q EuroSAT_RGB.zip -d data/

# split 70/15/15
import pathlib, random, shutil
root = pathlib.Path("data/EuroSAT_RGB")
splits = {"train":0.7, "val":0.15, "test":0.15}

for cls in root.iterdir():
    if not cls.is_dir(): continue
    imgs = list(cls.glob("*.jpg")); random.shuffle(imgs)
    n=len(imgs); idx=0
    for split, frac in splits.items():
        dest = pathlib.Path(f"data/EuroSAT/{split}/{cls.name}")
        dest.mkdir(parents=True, exist_ok=True)
        take = int(frac*n)
        for img in imgs[idx:idx+take]:
            shutil.copy(img, dest/img.name)
        idx += take
print("✓ EuroSAT split done")


✓ EuroSAT split done


In [3]:
# Cell 3 ─────────────────────────────────────────────────────────────
!unzip -q EuroSAT_Project.zip -d /content/
%cd /content/EuroSAT_Project
!ls src      # βλέπεις train.py, datasets.py, κ.λπ.


/content/EuroSAT_Project
datasets.py  evaluate.py  __init__.py  models.py  train.py  utils.py


In [6]:
%cd /content/EuroSAT_Project
!python src/train.py --data_dir ../data/EuroSAT --model efficientnet_b4 --epochs 2 --batch 128 --device cuda


/content/EuroSAT_Project
model.safetensors: 100% 77.9M/77.9M [00:00<00:00, 338MB/s]
/content/EuroSAT_Project/src/train.py:30: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
[1/2] train_loss=1.105 acc=63.24
[2/2] train_loss=0.452 acc=85.33


In [7]:
# Cell 5 ─────────────────────────────────────────────────────────────
!python src/train.py \
        --data_dir ../data/EuroSAT \
        --model efficientnet_b4 \
        --epochs 100 \
        --batch 256 \
        --device cuda


/content/EuroSAT_Project/src/train.py:30: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
[1/100] train_loss=1.388 acc=53.64
[2/100] train_loss=0.544 acc=82.39
[3/100] train_loss=0.368 acc=87.88
[4/100] train_loss=0.308 acc=89.90
[5/100] train_loss=0.264 acc=91.18
[6/100] train_loss=0.245 acc=92.04
[7/100] train_loss=0.218 acc=92.80
[8/100] train_loss=0.205 acc=93.21
[9/100] train_loss=0.198 acc=93.65
[10/100] train_loss=0.181 acc=94.05
[11/100] train_loss=0.170 acc=94.35
[12/100] train_loss=0.170 acc=94.58
[13/100] train_loss=0.158 acc=94.79
[14/100] train_loss=0.154 acc=94.95
Traceback (most recent call last):
  File "/content/EuroSAT_Project/src/train.py", line 52, in <module>
    main(args)
  File "/content/EuroSAT_Project/src/train.py", line 35, in main
    loss_meter.update(loss.item(), x.size(0))
                      ^^^^^^^^^^^
KeyboardInterrupt


In [8]:
!python src/train.py --model efficientnet_b4 --batch 256

[1/100] train_loss=1.388  train_acc=53.64  val_acc=82.42
[2/100] train_loss=0.541  train_acc=82.74  val_acc=91.56
[3/100] train_loss=0.372  train_acc=87.99  val_acc=93.60
[4/100] train_loss=0.311  train_acc=89.92  val_acc=94.79
[5/100] train_loss=0.266  train_acc=91.20  val_acc=95.48
[6/100] train_loss=0.242  train_acc=92.30  val_acc=96.00
[7/100] train_loss=0.225  train_acc=92.60  val_acc=96.42
[8/100] train_loss=0.216  train_acc=92.99  val_acc=96.42
[9/100] train_loss=0.197  train_acc=93.41  val_acc=96.79
[10/100] train_loss=0.180  train_acc=94.16  val_acc=96.77
[11/100] train_loss=0.179  train_acc=94.23  val_acc=96.59
[12/100] train_loss=0.163  train_acc=94.66  val_acc=96.84
[13/100] train_loss=0.152  train_acc=95.15  val_acc=97.14
[14/100] train_loss=0.145  train_acc=95.22  val_acc=97.19
[15/100] train_loss=0.144  train_acc=95.34  val_acc=97.23
[16/100] train_loss=0.140  train_acc=95.37  val_acc=97.53
[17/100] train_loss=0.138  train_acc=95.47  val_acc=97.48
[18/100] train_loss=0.1

In [9]:
!python src/train.py --model swin_tiny --batch 256

/usr/local/lib/python3.11/dist-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
model.safetensors: 100% 114M/114M [00:00<00:00, 338MB/s]
[1/100] train_loss=0.403  train_acc=87.66  val_acc=96.72
[2/100] train_loss=0.182  train_acc=94.16  val_acc=97.75
[3/100] train_loss=0.154  train_acc=95.13  val_acc=97.53
[4/100] train_loss=0.132  train_acc=95.93  val_acc=98.20
[5/100] train_loss=0.126  train_acc=95.90  val_acc=98.62
[6/100] train_loss=0.115  train_acc=96.39  val_acc=97.93
[7/100] train_loss=0.118  train_acc=96.34  val_acc=98.15
[8/100] train_loss=0.106  train_acc=96.62  val_acc=98.37
[9/100] train_loss=0.104  train_acc=96.60  val_acc=98.30
[10/100] train_loss=0.099  train_acc=96.83  val_acc=98.54
[11/100] train_loss=0.105  train_acc=96.77  val_acc=98

In [12]:
!python src/eval.py


efficientnet_b4: test accuracy 98.72%
/usr/local/lib/python3.11/dist-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
swin_tiny: test accuracy 99.19%


In [26]:
!python src/clean_matrix.py


efficientnet_b4: test accuracy 98.72%
/usr/local/lib/python3.11/dist-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
swin_tiny: test accuracy 99.19%
saved -> efficientnet_b4_cm_clean.png
saved -> swin_tiny_cm_clean.png
saved -> efficientnet_b4_qual.png
saved -> swin_tiny_qual.png


In [32]:
!zip -r EuroSAT_workspace.zip *

Streaming output truncated to the last 5000 lines.
  adding: data/EuroSAT_RGB/HerbaceousVegetation/HerbaceousVegetation_1792.jpg (deflated 3%)
  adding: data/EuroSAT_RGB/HerbaceousVegetation/HerbaceousVegetation_1330.jpg (deflated 3%)
  adding: data/EuroSAT_RGB/HerbaceousVegetation/HerbaceousVegetation_968.jpg (deflated 4%)
  adding: data/EuroSAT_RGB/HerbaceousVegetation/HerbaceousVegetation_2216.jpg (deflated 3%)
  adding: data/EuroSAT_RGB/HerbaceousVegetation/HerbaceousVegetation_2394.jpg (deflated 3%)
  adding: data/EuroSAT_RGB/HerbaceousVegetation/HerbaceousVegetation_166.jpg (deflated 2%)
  adding: data/EuroSAT_RGB/HerbaceousVegetation/HerbaceousVegetation_544.jpg (deflated 2%)
  adding: data/EuroSAT_RGB/HerbaceousVegetation/HerbaceousVegetation_1253.jpg (deflated 3%)
  adding: data/EuroSAT_RGB/HerbaceousVegetation/HerbaceousVegetation_2386.jpg (deflated 3%)
  adding: data/EuroSAT_RGB/HerbaceousVegetation/HerbaceousVegetation_1482.jpg (deflated 3%)
  adding: data/EuroSAT_RGB/Herba

In [31]:
ls

data/  EuroSAT_Project/  EuroSAT_Project.zip  EuroSAT_RGB.zip  sample_data/


In [29]:
!zip -r EuroSAT_all.zip checkpoints outputs reports


  adding: checkpoints/ (stored 0%)
  adding: checkpoints/efficientnet_b4_best.pth (deflated 7%)
  adding: checkpoints/swin_tiny_best.pth (deflated 7%)
  adding: outputs/ (stored 0%)
  adding: reports/ (stored 0%)
  adding: reports/part_I_report.docx (stored 0%)
  adding: reports/part_II_report.docx (stored 0%)
